#### 0. Imports

In [1]:
# Langchain
from langchain_community.agent_toolkits import GmailToolkit
from langchain_community.tools.gmail.search import GmailSearch
from langchain_community.tools.gmail.create_draft import GmailCreateDraft
from langchain_community.tools.gmail.get_thread import GmailGetThread
from langchain_community.tools.gmail.utils import build_resource_service, get_gmail_credentials
from langchain_community.llms import Ollama
from langchain.tools import tool

# Other
from textwrap import dedent

# CrewAi
from crewai import Agent, Task, Crew, Process

#### 1. LLM Settings

In [2]:
# llm settings
model = "model:latest"
host = "http://localhost:11434"
llm = Ollama(model = model, base_url = host, temperature = 0)

#### 2. Tools definition

In [3]:
class MailTools:
    @tool('Create Draft')
    def send_draft(data=""):
        """
        This tool creates a draft message to the specified subject.
        It receives the string "personemail|subject|message"
        """
        
        #  Expected output:
        # 'Draft created. Draft Id: r13453575123'  

        # check if agent message was not empty
        if data == "":
            return 'Draft not created.'

        # split data
        split = data.split('|')
        if len(split) != 3:
            return 'The string should be same exact format: "personemail|subject|message"'
            
        email, subject, message = split

        # create draft
        status = draft({
                'to': [email],
                'subject': subject,
                'message': message
        })
        
        return status

In [7]:
# instantiate draft
draft = GmailCreateDraft()

#### 3. Agent definition

In [8]:
def interact(task: str) -> str:
    assistant = Agent(
        role = 'You are a personal assistant.',
        goal = """
        Execute the tool to acomplish asked task and give the final 
        answer only if 'Draft created', keep the email message exact as requested.
    
        Your response must have-to:
        1 - Your message should have the same email asked.
        2 - You are allowed to taylor the subject and message.
        3 - You always answer in this exact Structure "personemail|subject|message"
        4 - NOT ALLOWED: data="..."
        """,
        backstory = "",
        verbose = False,
        allow_delegation = False,
        tools = [ MailTools.send_draft ],
        llm = llm
    )
    
    task1 = Task(
        description = task,
        agent = assistant
    )
    
    crew = Crew(
        agents = [assistant],
        tasks = [task1],
        verbose = 2,
        llm = llm
    )
    
    result = crew.kickoff()

#### Testing

In [9]:
interact("Send a Draft to barbara@nice.com explaining the reason why im not going with her. Barbara I'm not going because i got flu, I'm sorry. I hope you have funzies there!")

[DEBUG]: Working Agent: You are a personal assistant.
[INFO]: Starting Task: Send a Draft to barbara@nice.com explaining the reason why im not going with her. Barbara I'm not going because i got flu, I'm sorry. I hope you have funzies there!
[DEBUG]: [You are a personal assistant.] Task output: r-6992201393828731870<|im_end|>


